In [1]:
import os
import time
import datetime
from PIL import Image
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms

In [3]:
print(dir(torchvision.models))

['AlexNet', 'DenseNet', 'GoogLeNet', 'GoogLeNetOutputs', 'Inception3', 'InceptionOutputs', 'MNASNet', 'MobileNetV2', 'ResNet', 'ShuffleNetV2', 'SqueezeNet', 'VGG', '_GoogLeNetOutputs', '_InceptionOutputs', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_utils', 'alexnet', 'densenet', 'densenet121', 'densenet161', 'densenet169', 'densenet201', 'detection', 'googlenet', 'inception', 'inception_v3', 'mnasnet', 'mnasnet0_5', 'mnasnet0_75', 'mnasnet1_0', 'mnasnet1_3', 'mobilenet', 'mobilenet_v2', 'quantization', 'resnet', 'resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50', 'resnext101_32x8d', 'resnext50_32x4d', 'segmentation', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'shufflenetv2', 'squeezenet', 'squeezenet1_0', 'squeezenet1_1', 'utils', 'vgg', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn', 'video', 'wide_resnet101_2', 'wide_r

In [2]:
cuda_gpu = torch.cuda.is_available()
if cuda_gpu:
    device = torch.device("cuda:0")
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu:0")
nclass = 100

In [ ]:
torchvision.models.inception_v3

In [3]:
model = torchvision.models.densenet201(pretrained=True)
model.classifier = nn.Linear(1920, 100, bias=True)
model = model.to(device)

In [4]:
for name, value in model.named_parameters():
    if "fc" not in name and "classifier" not in name:
        value.requires_grad = False
# for name, param in model.named_parameters():
#     print(name,param.requires_grad)

In [5]:
EPOCHS = 30
BATCH_SIZE = 128
LR = 0.005
SAVE_EPOCH = 3
VAL_EPOCH = 6
verbose = 250
loss_func = nn.CrossEntropyLoss()

In [6]:
data_transformer = transforms.Compose([
    #transforms.RandomResizedCrop(64),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(256),
    transforms.ToTensor()
    #transforms.Normalize([124,117,104],[58.4,57.1,57.4])
    ])

class ImageNetData(Dataset):
    
    def __init__(self, path, info, nclass):
        self.path = path
        info_path = os.path.join(path, info)
        f = open(info_path)
        self.data = f.read().splitlines()
        self.transforms = data_transformer
        self.nclass = nclass
    
    def __len__(self):
        return len(self.data)
        
        
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.path, self.data[index].split(' ')[0]))
        img = self.transforms(img)
        label = self.data[index].split(' ')[1]
        label = int(label)
#         one_hot = np.zeros(self.nclass, dtype=np.int)
#         one_hot[label] = 1
        return (img, label)

path = "D:/课程文件/大四上/深度学习导论/作业/5/TinyImageNet/TinyImageNet"
train_info_path = path + '/train.txt'
val_info_path = path + '/val.txt'
train_set = ImageNetData(path, "train.txt", nclass)
val_set = ImageNetData(path, "val.txt", nclass)

train_loader = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=BATCH_SIZE)      

In [7]:
ckptpath = "./ckpts/"
ckpts = sorted([x for x in os.listdir(ckptpath) if x.endswith(".ckpt")])
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
lr_reduct = lambda epochs: 0.5 ** (epochs // 20)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_reduct)
if len(ckpts) >= 1:
    state_dicts = torch.load(ckptpath + ckpts[-1])
    model.load_state_dict(state_dicts['model'])
    optimizer.load_state_dict(state_dicts['optimizer'])
    scheduler.load_state_dict(state_dicts['scheduler'])

In [8]:
train_loss = []
train_acc = []
val_loss = []
val_acc = []
with torch.autograd.set_detect_anomaly(True):
    for epochs in range(1, EPOCHS + 1):
        print('Epoch {}/{}'.format(epochs, EPOCHS))
        start_time = time.time()
        running_train_loss = []
        running_train_acc = []
        for i, data in enumerate(train_loader):
            inputs = data[0].to(device)
            targets = data[1].to(device)
            outputs = model(inputs)
            loss = loss_func(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pred = outputs.argmax(axis=1)
            running_train_loss.append(float(loss))
            running_train_acc.append(float((pred==targets).sum()) / BATCH_SIZE)
            if verbose > 0 and (i + 1) % verbose == 0:
                print("Train: {}/{} - Average Time: {}".format(i + 1, len(train_loader), (time.time()-start_time) / (i + 1)))
        print("End of Train Epoch: {} - Average Time: {}".format(epochs, (time.time() - start_time) / (i + 1)))
        # 周期性清除CUDA缓存
        torch.cuda.empty_cache()
        if epochs % SAVE_EPOCH == 0:
            torch.save({'model': model.state_dict(), 
                       'optimizer': optimizer.state_dict(), 
                       'scheduler': scheduler.state_dict()},
                       ckptpath + "model_{}.ckpt".format(int(datetime.datetime.now().timestamp())))
        if epochs % VAL_EPOCH == 0:
            running_val_loss = []
            running_val_acc = []
            for i, data in enumerate(val_loader):
                inputs=data[0].to(device)
                targets=data[1].to(device)
                with torch.no_grad():
                    outputs = model(inputs)
                    loss = loss_func(outputs, targets)
                    pred = outputs.argmax(axis=1)
                    running_val_loss.append(float(loss))
                    running_val_acc.append(float((pred==targets).sum()) / BATCH_SIZE)
                if verbose > 0 and(i + 1) % verbose == 0:
                    print("Val: {}/{}".format(i + 1, len(val_loader)))
        
        scheduler.step()    
        train_loss.append(sum(running_train_loss) / len(running_train_loss))
        train_acc.append(sum(running_train_acc) / len(running_train_acc))
        duration = time.time() - start_time
        print('Train loss : {} Train accuracy : {}'.format(train_loss[-1], train_acc[-1]))
        if epochs % VAL_EPOCH == 0:
            val_loss.append(sum(running_val_loss) / len(running_val_loss))
            val_acc.append(sum(running_val_acc) / len(running_val_acc))
            print('Val   loss : {} Val   accuracy : {}'.format(val_loss[-1], val_acc[-1]))
        print('{:.0f}m {:.0f}s'.format(duration // 60, duration % 60))

Epoch 1/30
Train: 250/782 - Average Time: 1.8564409418106078
Train: 500/782 - Average Time: 1.8315183520317078
Train: 750/782 - Average Time: 1.8208494523366292
End of Train Epoch: 1 - Average Time: 1.8218429381280299
Train loss : 1.9640066785275783 Train accuracy : 0.8478760390025576
23m 45s
Epoch 2/30
Train: 250/782 - Average Time: 1.8566275329589843
Train: 500/782 - Average Time: 1.8193567266464234
Train: 750/782 - Average Time: 1.8153198652267457
End of Train Epoch: 2 - Average Time: 1.8108564926230388
Train loss : 1.945189712678685 Train accuracy : 0.8476862212276215
23m 36s
Epoch 3/30
Train: 250/782 - Average Time: 1.7821475791931152
Train: 500/782 - Average Time: 1.8213007016181946
Train: 750/782 - Average Time: 1.8398196277618408
End of Train Epoch: 3 - Average Time: 1.8393306524857231
Train loss : 1.9410330366600506 Train accuracy : 0.8468370364450127
23m 59s
Epoch 4/30


KeyboardInterrupt: 

In [ ]:
torch.save({'model': model.state_dict(), 
            'optimizer': optimizer.state_dict(), 
            'scheduler': scheduler.state_dict()},
            ckptpath + "model_{}.ckpt".format(int(datetime.datetime.now().timestamp())))

In [8]:
class ImageNetDataTest(Dataset):
    def __init__(self, path, info, nclass):
        self.path = path
        info_path = os.path.join(path, info)
        f = open(info_path)
        self.data = f.read().splitlines()
        self.transforms = data_transformer
        self.nclass = nclass
    
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.path, self.data[index].split(' ')[0]))
        img = self.transforms(img)
#         one_hot = np.zeros(self.nclass, dtype=np.int)
#         one_hot[label] = 1
        return (self.data[index], img)

test_info_path = path + '/test.txt'
test_set = ImageNetDataTest(path, "test.txt", nclass)
test_loader = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=False)

In [13]:
names = []
preds = []
preds_val = {}
for i, data in enumerate(test_loader):
    with torch.no_grad():
        name = data[0]
        inputs = data[1].to(device)
        outputs = model(inputs)
        preds_val[name] = outputs.cpu().numpy()
        pred = outputs.argmax(axis=1).squeeze()
        names.extend(name)
        preds.extend(pred.cpu().numpy())

In [14]:
df = pd.DataFrame([names,preds]).transpose()
df.columns = ['Id', 'Category']
df.loc[:, 'Id']=df['Id'].apply(lambda x: x.split('/')[-1])

In [15]:
df.to_csv("submission_densenet.csv", index=False)

In [16]:
import pickle
f = open("prediction_densenet.pkl", "wb")
pickle.dump(preds_val, f)
f.close()

In [9]:
time.localtime(1604911584)

time.struct_time(tm_year=2020, tm_mon=11, tm_mday=9, tm_hour=16, tm_min=46, tm_sec=24, tm_wday=0, tm_yday=314, tm_isdst=0)

In [8]:
def getPrediction(loader, fname, verbose, total):
    preds_val = []
    with torch.no_grad():
        for i, data in enumerate(loader):
            inputs = data[0].to(device)
            targets = data[1].to(device)
            outputs = model(inputs)
            preds_val.append((targets.cpu().numpy(), outputs.cpu().numpy()))
            if (i + 1) % verbose == 0:
                print("Predict: {}/{}".format(i + 1, total))
            torch.cuda.empty_cache()
    f = open(fname, "wb")
    pickle.dump(preds_val, f)
    f.close()

train_loader = DataLoader(dataset=train_set, batch_size=64, shuffle=False)
val_loader = DataLoader(dataset=val_set, batch_size=64, shuffle=False)

In [9]:
getPrediction(train_loader, "prediction_densenet_train.pkl", 100, int(len(train_set)/64))

Predict: 100/1562
Predict: 200/1562
Predict: 300/1562
Predict: 400/1562
Predict: 500/1562
Predict: 600/1562
Predict: 700/1562
Predict: 800/1562
Predict: 900/1562
Predict: 1000/1562
Predict: 1100/1562
Predict: 1200/1562
Predict: 1300/1562
Predict: 1400/1562
Predict: 1500/1562


In [10]:
getPrediction(val_loader, "prediction_densenet_val.pkl", 100, int(len(val_set)/64))

Predict: 100/156
